In [21]:
#coding:utf-8
__author__ = 'Mr.数据杨'
__explain__ = '爬虫数据存储mongodb模板' \

import requests
from bs4 import BeautifulSoup
import pymongo

In [18]:
# 获取mongoClient对象
client = pymongo.MongoClient("localhost", 27017)
# 获取使用的database对象
db = client["TestNews"]

In [19]:
url = "http://www.cwestc.com/MroeNews.aspx?gd=1"
html = requests.get(url)
soup = BeautifulSoup(html.text,'lxml')

In [20]:
lptable = soup.find('table',width='780')
for i in lptable.find_all("td",width="680"): 
    Title = i.b.strong.a.string  # title字段
    Url =  "http://www.cwestc.com"+i.find('a')['href'] # url字段
    Date =  Url.split("/")[4]  # 日期字段
    # 根据url抓取正文
    content = requests.get(Url)
    content.encoding = "utf-8"
    soup = BeautifulSoup(content.text,'html.parser')
    try:
        NewsBody = str(soup.find('td',{'class':'newsContent'}))
    except:
        NewsBody = "无内容"
    #写入Mongodb
    one_page={'title':Title,'url':Url,'date':Date,"NewsBody":NewsBody}
    db["TestNews"].insert_one(one_page)